<a href="https://colab.research.google.com/github/sands-eg/GEE_training/blob/main/PLANET.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classificating PLANET data

In [2]:
import ee

In [3]:
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=9HNF_mUOMYElN_LfY4-xuN1peabvHsLMVd6myIt6dhU&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWja1mxY5MqKTQbzUkneQa85FeEHBiamgtuHkxj658R34zcb8OSRokc

Successfully saved authorization token.


In [4]:
# loading an image 
first = ee.ImageCollection("projects/planet-nicfi/assets/basemaps/asia").filterBounds(ee.Geometry.Rectangle([[117.586,0.668],[117.756,0.805]])).filterDate('2019-01-01', '2019-12-31').first()      

In [6]:
from IPython.display import Image

Image(url=first.getThumbUrl({'min': 64, 'max': 5454, 'bands':['R','G','B'],'dimensions': 512}))